# Import

In [1]:
from dotenv import load_dotenv
import os
import mlflow
from typing import List, Dict, Optional

from collections import defaultdict
from dataclasses import dataclass

os.environ["no_proxy"] = "*"

mlflow.set_tracking_uri("http://198.215.61.34:8153/")
mlflow.set_experiment("s440708_Shiqiu")
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

print("Loaded API key:", AZURE_OPENAI_KEY[:5] + "..." if AZURE_OPENAI_KEY else "MISSING")

Loaded API key: FJ5Gr...


# Config

In [35]:
data_config = {
    "dataset_path": "/work/bioinformatics/s440708/MODULE_3_MATERIALS/data/7B_golden.json",  
    "output_path": "/work/bioinformatics/s440708/MODULE_3_MATERIALS/outputs/",
    "dataset_name": "bioasq"
}

model_config = {           
    "model_name": AZURE_OPENAI_DEPLOYMENT_NAME,     
    #"model_name": "qwen3:4b",   
    'frequency_penalty': 0.0,
    'presence_penalty': 0.0,
    "temperature": 1.,
    "max_tokens": 800,
    "top_p": 1.0,
    
}


# Load data

In [12]:

import json

def load_geneturing(path: str) -> List[Dict[str, str]]:
    with open(path, "r") as f:
        raw_data = json.load(f)

    flat_data = []
    counter = 0
    for question, answers in raw_data.items():
        for dicts in answers:
    
            flat_data.append({
                "task": dicts['type'],
                'id': counter,
                "question": dicts['body'],
                "answer": dicts['ideal_answer'][0]
            })
            counter += 1
    

    return flat_data

# config = Config()
dataset = load_geneturing(data_config['dataset_path'])
print(f"Loaded {len(dataset)} examples from {data_config['dataset_path']}")


Loaded 500 examples from /work/bioinformatics/s440708/MODULE_3_MATERIALS/data/7B_golden.json


In [13]:
# 3.3 Create the pandas dataframe from the collection of rows

import pandas as pd

def build_dataframe(flat_data: List[Dict[str, str]]) -> pd.DataFrame:

    df = pd.DataFrame(flat_data)
    
    df = df[["id", "task", "question", "answer"]]  
    df.set_index("id", inplace=True)
    return df

df = build_dataframe(dataset)
print(f"DataFrame shape: {df.shape}")
display(df.head())

DataFrame shape: (500, 3)


,task,question,answer
id,,,
0,factoid,What does the pembrolizumab companion diagnost...,Administration of pembrolizumab requires a com...
1,factoid,What is the combined effect of Nfat and miR-25?,Increased calcineurin/Nfat signalling and decr...
2,list,In what states are GDF15 expression increased?,Growth differentiation factor 15 (GDF-15) is e...
3,factoid,Which genomic positions are preferentially sel...,Preferential integration occurs in non-coding ...
4,factoid,Which Lisp framework has been developed for im...,FunImageJ is a Lisp framework for scientific i...


# Create a Structured Output

In [14]:
system_message = (
    "You are a genomics assistant that returns structured answers. "
    "You will be provided with a gene-related question and should return your answer "
    "in structured JSON format following a schema including task, question, answer, and explanation."
)

In [37]:
from pydantic import BaseModel, Field
from typing import Optional

import os
import pprint as pp
from openai import AzureOpenAI

class GeneHopAnswer(BaseModel):
    """
    Structured answer format for gene-level LLM QA.
    """
    task: str = Field(..., description="GeneHop task name, e.g., 'Gene alias'")
    question: str = Field(..., description="The user query")
    answer: str = Field(..., description="Structured, atomic answer returned by the model")
    explanation: Optional[str] = Field(
        None,
        description="Optional reasoning or justification provided by the model"
    )



query = {
    "task": "SNP gene function",
    "question": "What is the function of the gene associated with SNP rs1217074595? Let's decompose the question to sub-questions and solve them step by step.",
}

prompt = f"""Here is a question: "{query['question']}". 
Return the answer in valid JSON format matching this schema: 
{{"task": str, "question": str, "answer": str, "explanation": Optional[str]}}.
The 'task' should be '{query['task']}'.
"""

# few_shot_examples = [
#     {"role": "user", "content": "What are the aliases of the gene that contains this sequnece:"
#      "ACTTCCAACATGGCGGCGGCCGGGGCGGCGGTGGCGCGCAGCCCGGGAATCGGAGCGGGACCTGCGCTGAGAGCCCGGCGCTCGCCCCCGCCGCGGGCCGCACGGCTGCCGCG."
#      "Let's decompose the question to sub-questions and solve them step by step."},
#     {"role": "assistant", "content": "QSOX2, SOXN, QSCN6L1"},
#     {"role": "user", "content": "List chromosome locations of the genes related to Hemolytic anemia due to phosphofructokinase deficiency. Let's decompose the question to sub-questions and solve them step by step."},
#     {"role": "assistant", "content": "21q22.3"},
#     {"role": "user", "content": "What is the function of the gene associated with SNP rs1241371358? Let's decompose the question to sub-questions and solve them step by step."},
#     {"role": "assistant", "content": "Predicted to be active in cytosol."}
# ]

messages = [{"role": "system", "content": system_message},
                {"role": "user", "content": prompt}]



client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-08-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    timeout=10,
    max_retries=3,
)

response = client.beta.chat.completions.parse(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages=messages,
    temperature=0,
    max_tokens=1024,
    response_format=GeneHopAnswer,
)

pp.pprint(response.model_dump())
# from ollama import chat
# response = chat(
#   messages=messages,
#   model= model_config['model_name'],
#   format=GeneHopAnswer.model_json_schema(),
# )

# genehop_answer = GeneHopAnswer.model_validate_json(response.message.content)
# answer = genehop_answer.model_dump()
# pp.pprint(answer)




{'choices': [{'content_filter_results': {'hate': {'filtered': False,
                                                  'severity': 'safe'},
                                         'protected_material_code': {'detected': False,
                                                                     'filtered': False},
                                         'protected_material_text': {'detected': False,
                                                                     'filtered': False},
                                         'self_harm': {'filtered': False,
                                                       'severity': 'safe'},
                                         'sexual': {'filtered': False,
                                                    'severity': 'safe'},
                                         'violence': {'filtered': False,
                                                      'severity': 'safe'}},
              'finish_reason': 'stop',
              'index': 0

# Create Query Model Function

In [41]:
def query_model(
    system_message: dict,
    user_query: str,
) -> str:
    from ollama import chat
    
    prompt = f"""Here is a question: "{user_query['question']}". 
    Return the answer in valid JSON format matching this schema: 
    {{"task": str, "question": str, "answer": str, "explanation": Optional[str]}}.
    The 'task' should be '{user_query['task']}'.
    """

    messages = [{"role": "system", "content": system_message},
                {"role": "user", "content": prompt}]


    # response = chat(
    #     messages=messages,
    #     model= model_config['model_name'],
    #     format=GeneHopAnswer.model_json_schema(),
    #     )
    response = client.beta.chat.completions.parse(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages=messages,
    temperature=0,
    max_tokens=1024,
    response_format=GeneHopAnswer,
    )
    answer = response.model_dump()
    # genehop_answer = GeneHopAnswer.model_validate_json(response.message.content)
    # answer = genehop_answer.model_dump()
    return answer

In [42]:
from tqdm import tqdm
from typing import Any, Callable, Dict, List, Optional, Union, cast
import json

def collect_structured_output(
    # client: Any,
    dataset: List[Dict[str, str]],
    system_message: Dict[str, str],
) -> List[Dict[str, str]]:

    answers = []

    with mlflow.start_run(run_name="geneturing-eval"):
        mlflow.log_param("model", model_config["model_name"])
        mlflow.log_param("temperature", model_config["temperature"])
        mlflow.log_param("max_tokens", model_config["max_tokens"])
        mlflow.log_param("top_p", model_config["top_p"])

        for item in tqdm(dataset, desc="Evaluating"):
            item_id = item.get("id")
            task = item.get("task")
            question = item.get("question")
           
            #gold_answer = item.get("answer")
            query = {'task': task, "question": question}
            print(query)
            # try:

            answer = query_model(
                system_message, query
            )
            answer['success'] = True
            answer['id'] = item_id
                

            # except Exception:
            #     answer = {}
            #     answer['success'] = False
            #     answer['id'] = item_id
            #     answer['task'] = task
            #     answer['question'] = question
            #     answer['answer'] = ""
            #     answer['explanation'] = ""
        

            answers.append(answer)

        # Calculate and log metrics
        df = pd.DataFrame(answers)
        success_rate = df["success"].mean()
        print(success_rate)
        #valid_scores_df = df[df["score"].notna()]
        #overall_score = valid_scores_df["score"].mean()

        mlflow.log_metric("success_rate", success_rate)
        #mlflow.log_metric("overall_score", overall_score)

        # for task_name, task_df in df.groupby("task"):
        #     mlflow.log_metric(
        #         f"score_{task_name.replace(' ', '_')}", task_df["score"].mean()
        #     )

        # Save to json and log as artifact
        result_json_path = os.path.join(data_config['output_path'], f"{data_config['dataset_name']}_results.csv")
        with open(result_json_path, 'w') as f:
            json.dump(answers, f, indent=2)
      
        # df.to_csv(result_csv_path, index=False)
        mlflow.log_artifact(result_json_path)

    return answers

In [43]:
results = collect_structured_output(dataset, system_message[0])

Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

{'task': 'factoid', 'question': 'What does the pembrolizumab companion diagnostic test assess?'}


Evaluating:   0%|          | 1/500 [00:01<11:12,  1.35s/it]

{'task': 'factoid', 'question': 'What is the combined effect of Nfat and miR-25?'}


Evaluating:   0%|          | 2/500 [00:02<09:55,  1.20s/it]

...

{'task': 'yesno', 'question': 'Is TNF-α an activator of pancreatic stellate cells?'}


Evaluating: 100%|█████████▉| 499/500 [10:21<00:01,  1.13s/it]

{'task': 'yesno', 'question': 'Can mitochondria transfer from cell to cell?'}


Evaluating: 100%|██████████| 500/500 [10:22<00:00,  1.24s/it]

1.0
🏃 View run geneturing-eval at: http://198.215.61.34:8153/#/experiments/9/runs/fdbb75bc3067446ea60fa7638a523c63
🧪 View experiment at: http://198.215.61.34:8153/#/experiments/9


# calculate cosine similarity

In [ ]:
import json
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import os

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )


tokenizer = AutoTokenizer.from_pretrained("FremyCompany/BioLORD-2023")
model = AutoModel.from_pretrained("FremyCompany/BioLORD-2023")


with open("/work/bioinformatics/s440708/MODULE_3_MATERIALS/outputs/bioasq_results.csv", "r") as f:
    pred_data = json.load(f)

with open("/work/bioinformatics/s440708/MODULE_3_MATERIALS/data/7B_golden.json", "r") as f:
    gold_data = json.load(f)


results = []
for entry in pred_data:
    question = entry['question']
    pred_answer = entry['answer']

    found = False
    for task, qa_dict in gold_data.items():
        for gold_q, gold_a_list in qa_dict.items():
            if gold_q.strip() == question.strip():
                found = True
                for gold_answer in gold_a_list:
                   
                    inputs = tokenizer([pred_answer, gold_answer], padding=True, truncation=True, return_tensors="pt")
                    with torch.no_grad():
                        outputs = model(**inputs)
                    embeddings = mean_pooling(outputs, inputs['attention_mask'])
                    embeddings = F.normalize(embeddings, p=2, dim=1)
                    cosine_sim = F.cosine_similarity(embeddings[0], embeddings[1], dim=0).item()
                    results.append({
                        'question': question,
                        'predicted_answer': pred_answer,
                        'gold_answer': gold_answer,
                        'cosine_similarity': cosine_sim
                    })
    if not found:
        results.append({
            'question': question,
            'predicted_answer': pred_answer,
            'gold_answer': None,
            'cosine_similarity': None,
            'note': 'No gold answer found'
        })

import pandas as pd
df = pd.DataFrame(results)


In [ ]:
import json
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from tqdm import tqdm
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("FremyCompany/BioLORD-2023")
model = AutoModel.from_pretrained("FremyCompany/BioLORD-2023")

# Helper function for mean pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

# Load gold data
with open("/work/bioinformatics/s440708/MODULE_3_MATERIALS/data/genehop.json", "r") as f:
    gold_data = json.load(f)

# Load predictions
with open("/work/bioinformatics/s440708/MODULE_3_MATERIALS/outputs/genehop_results.csv", "r") as f:
    pred_data = json.load(f)

results = []
gold_entries = []
for task_name, task_dict in gold_data.items():
    
    for question, answers in task_dict.items():
            gold_entries.append({
                "task": task_name,
                "question": question.replace("Let's decompose the question to sub-questions and solve them step by step.","").strip(),
                "gold_answers": answers
            })

# Compute similarity
for i, pred_entry in enumerate(pred_data):
    question = pred_entry['question'].strip()
    predicted_answer = pred_entry['answer'].strip()
    task_name = pred_entry['task']

    match = gold_entries[i]
    gold_answer = match['gold_answers']
    if isinstance(gold_answer, list):
        gold_answer = " ".join(gold_answer)

   
    inputs = tokenizer([predicted_answer, gold_answer], padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = mean_pooling(outputs, inputs['attention_mask'])
        embeddings = F.normalize(embeddings, p=2, dim=1)
        cosine_sim = F.cosine_similarity(embeddings[0], embeddings[1], dim=0).item()
        print(cosine_sim)
        results.append({
            "task": task_name,
            "question": question,
            "predicted_answer": predicted_answer,
            "gold_answer": gold_answer,
            "cosine_similarity": cosine_sim
        })
    
   

df = pd.DataFrame(results)



100%|██████████| 3/3 [00:00<00:00, 5013.11it/s]


0.22228483855724335
0.12435595691204071
0.2232302576303482
0.17944329977035522
0.241394504904747
0.3022831976413727
0.3317318558692932
0.22889623045921326
0.2430797666311264
0.44673436880111694
0.5258044004440308
0.31004586815834045
0.20812499523162842
0.26675739884376526
0.256971538066864
0.16082169115543365
0.24365420639514923
0.17847153544425964
0.28906893730163574
0.6071949005126953
0.02541375160217285
0.5200710296630859
0.1370559185743332
0.34150242805480957
0.19680416584014893
0.1015663743019104
0.37616950273513794
0.3683997690677643
0.17296752333641052
0.27167296409606934
0.37701550126075745
0.4135938286781311
0.24612268805503845
0.16213443875312805
0.3622819781303406
0.31128329038619995
0.15481258928775787
0.27907854318618774
0.2902039587497711
0.39749252796173096
0.37894707918167114
0.2619859576225281
0.3804163932800293
0.16254813969135284
0.18180790543556213
0.38003015518188477
0.29259175062179565
0.39297351241111755
0.1963275521993637
0.17553435266017914
0.2015819549560547
0

In [ ]:
df['cosine_similarity'].mean()


np.float64(0.2703053500503302)

In [36]:

df.to_csv(os.path.join(data_config['output_path'], "genehop_ollama_results.csv"))

# 